In [43]:
import pandas as pd
#!pip install yfinance
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.optimize import minimize, Bounds, LinearConstraint

In [11]:
start_date = "2017-01-03"
end_date = "2022-07-29"
tickerlist = ['RY.TO','TD.TO','BNS.TO','CM.TO','CWB.TO','HCG.TO','LB.TO','NA.TO','BMO.TO']

In [12]:
tickerstr = ''.join(tickerlist)
data = yf.download(tickers=tickerlist, start=start_date, end=end_date, interval="1d", group_by='ticker')

[*********************100%***********************]  9 of 9 completed


In [13]:
df=pd.DataFrame(index=data.index)
for x in tickerlist:
    df = pd.merge(df, data[x]['Adj Close'].rename(x), left_index=True, right_index=True)

df=df.dropna(axis=0)

                 RY.TO      TD.TO     BNS.TO      CM.TO     CWB.TO     HCG.TO  \
Date                                                                            
2017-01-03   73.316238  53.386005  58.153591  42.166840  25.346199  31.204655   
2017-01-04   74.389961  53.976433  58.644836  42.478794  25.766714  31.263548   
2017-01-05   74.373917  54.104084  58.775322  42.535858  25.717239  30.988705   
2017-01-06   74.293793  54.176483  58.790668  42.490208  25.741980  30.704044   
2017-01-09   74.133553  53.927128  58.606457  42.353256  25.387428  29.977671   
...                ...        ...        ...        ...        ...        ...   
2022-07-22  122.339996  82.029999  75.089996  62.959999  25.299999  25.150000   
2022-07-25  123.099998  82.470001  75.739998  63.290001  25.600000  25.620001   
2022-07-26  121.849998  81.400002  75.860001  63.049999  25.209999  25.190001   
2022-07-27  122.650002  82.440002  76.699997  63.680000  25.389999  25.540001   
2022-07-28  123.750000  82.0

In [13]:
df_pctchg = df.pct_change()

In [14]:
print(df_pctchg)

               RY.TO     TD.TO    BNS.TO     CM.TO    CWB.TO    HCG.TO  \
Date                                                                     
2017-01-03       NaN       NaN       NaN       NaN       NaN       NaN   
2017-01-04  0.014645  0.011060  0.008447  0.007398  0.016591  0.001887   
2017-01-05 -0.000216  0.002365  0.002225  0.001343 -0.001920 -0.008791   
2017-01-06 -0.001077  0.001338  0.000261 -0.001073  0.000962 -0.009186   
2017-01-09 -0.002157 -0.004603 -0.003133 -0.003223 -0.013773 -0.023657   
...              ...       ...       ...       ...       ...       ...   
2022-07-22 -0.003707 -0.002553 -0.003451  0.004147 -0.009397  0.003992   
2022-07-25  0.006212  0.005364  0.008656  0.005241  0.011858  0.018688   
2022-07-26 -0.010154 -0.012974  0.001584 -0.003792 -0.015234 -0.016784   
2022-07-27  0.006565  0.012776  0.011073  0.009992  0.007140  0.013894   
2022-07-28  0.008969 -0.004488  0.006519  0.007538  0.003939  0.011355   

               LB.TO     NA.TO    BMO

In [15]:
corr = df_pctchg.corr()

           RY.TO     TD.TO    BNS.TO     CM.TO    CWB.TO    HCG.TO     LB.TO  \
RY.TO   1.000000  0.850590  0.842268  0.813709  0.606172  0.303547  0.625138   
TD.TO   0.850590  1.000000  0.865843  0.818254  0.641601  0.317292  0.658345   
BNS.TO  0.842268  0.865843  1.000000  0.834102  0.638694  0.323147  0.672760   
CM.TO   0.813709  0.818254  0.834102  1.000000  0.640523  0.333083  0.663341   
CWB.TO  0.606172  0.641601  0.638694  0.640523  1.000000  0.388507  0.627081   
HCG.TO  0.303547  0.317292  0.323147  0.333083  0.388507  1.000000  0.290011   
LB.TO   0.625138  0.658345  0.672760  0.663341  0.627081  0.290011  1.000000   
NA.TO   0.807352  0.807550  0.824888  0.808224  0.661559  0.348647  0.649697   
BMO.TO  0.835289  0.843696  0.857003  0.857002  0.664418  0.331414  0.675475   

           NA.TO    BMO.TO  
RY.TO   0.807352  0.835289  
TD.TO   0.807550  0.843696  
BNS.TO  0.824888  0.857003  
CM.TO   0.808224  0.857002  
CWB.TO  0.661559  0.664418  
HCG.TO  0.348647  0.33141

In [105]:
T = 3
dt = 0.001
S = 1000
sigma = 0.2218
r = 0.0261

u = math.exp(sigma*math.sqrt(dt))
d = 1/u
p = (math.exp(r*dt) - d) / (u - d)
N = (int)(T/dt)

participation = 0.2
floor = 0
ceil = 10000000
var_return = 0
early_excercise = 2

binomialtree = [[]]*N
binomialtree[0] = [S]

for counter1 in range(1,N):
    binomialtree[counter1] = [0]*(counter1+1)
    binomialtree[counter1][0] = binomialtree[counter1-1][0]*d
    for counter2 in range(1,counter1+1):
        binomialtree[counter1][counter2] = binomialtree[counter1-1][counter2-1]*u
        
valuationtree = [[]]*N
valuationtree[N-1] = [0]*N

for counter1 in range (N):
    var_return = math.log(binomialtree[N-1][counter1] / S)
    var_return = var_return * participation
    var_return = min(var_return, ceil)
    var_return = max(var_return, floor)
    valuationtree[N-1][counter1] = S*(var_return+1)

for counter1 in range(N-1,0,-1):
    valuationtree[counter1-1] = [0]*(counter1)
    for counter2 in range(0,counter1):
        valuationtree[counter1-1][counter2] = math.exp(-r*dt)*(p*valuationtree[counter1][counter2]+(1-p)*valuationtree[counter1][counter2+1])
        if (counter1 == (int)(early_excercise/dt)):
            var_return = math.log(binomialtree[counter1-1][counter2] / S)
            var_return = var_return * participation
            var_return = min(var_return, ceil)
            var_return = max(var_return, floor)
            
            if (S*(math.exp(-r*T)+var_return) > valuationtree[counter1-1][counter2]):
                print ("Early Excercise at:" + str(counter2))
            
            valuationtree[counter1-1][counter2] = max(S*(math.exp(-r*T)+var_return),valuationtree[counter1-1][counter2])
            

print(valuationtree[0][0])

Early Excercise at:1682
Early Excercise at:1683
Early Excercise at:1684
Early Excercise at:1685
Early Excercise at:1686
Early Excercise at:1687
Early Excercise at:1688
Early Excercise at:1689
Early Excercise at:1690
Early Excercise at:1691
Early Excercise at:1692
Early Excercise at:1693
Early Excercise at:1694
Early Excercise at:1695
Early Excercise at:1696
Early Excercise at:1697
Early Excercise at:1698
Early Excercise at:1699
Early Excercise at:1700
Early Excercise at:1701
Early Excercise at:1702
Early Excercise at:1703
Early Excercise at:1704
Early Excercise at:1705
Early Excercise at:1706
Early Excercise at:1707
Early Excercise at:1708
Early Excercise at:1709
Early Excercise at:1710
Early Excercise at:1711
Early Excercise at:1712
Early Excercise at:1713
Early Excercise at:1714
Early Excercise at:1715
Early Excercise at:1716
Early Excercise at:1717
Early Excercise at:1718
Early Excercise at:1719
Early Excercise at:1720
Early Excercise at:1721
Early Excercise at:1722
Early Excercise 